In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from utils import mnist

In [2]:
train_loader, test_loader = mnist()

In [3]:
class Net(nn.Module):
    def __init__(self, log_softmax=False):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.log_softmax = log_softmax
        self.optim = optim.SGD(self.parameters(), lr=1.0)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        if self.log_softmax:
            x = F.log_softmax(x, dim=1)
        else:
            x = torch.log(F.softmax(x, dim=1))
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.nll_loss(output, target, **kwargs)
        return self._loss

In [4]:
def train(epoch, models):
    for batch_idx, (data, target) in enumerate(train_loader):
        for model in models:
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
        print(line + losses)

In [5]:
models = [Net(), Net(True)]

In [6]:
avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, len(test_loader.dataset), p)
line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

def test(models):
    test_loss = [0]*len(models)
    correct = [0]*len(models)
    with torch.no_grad():
        for data, target in test_loader:
            output = [m(data) for m in models]
            for i, m in enumerate(models):
                test_loss[i] += m.loss(output[i], target, reduction='sum').item() # sum up batch loss
                pred = output[i].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    for i in range(len(models)):
        test_loss[i] /= len(test_loader.dataset)
    correct_pct = [100. * c / len(test_loader.dataset) for c in correct]
    lines = '\n'.join([line(i, test_loss[i], correct[i], correct_pct[i]) for i in range(len(models))]) + '\n'
    report = 'Test set:\n' + lines
    
    print(report)

In [7]:
for epoch in range(1, 3):
    train(epoch, models)
    test(models)

Train Epoch: 1 [0/60000 (0%)]	Losses 0: 2.439889 1: 2.297994
Train Epoch: 1 [10000/60000 (17%)]	Losses 0: 0.158167 1: 0.143543
Train Epoch: 1 [20000/60000 (33%)]	Losses 0: 0.170912 1: 0.196813
Train Epoch: 1 [30000/60000 (50%)]	Losses 0: 0.185871 1: 0.225315
Train Epoch: 1 [40000/60000 (67%)]	Losses 0: 0.274120 1: 0.255598
Train Epoch: 1 [50000/60000 (83%)]	Losses 0: 0.348655 1: 0.320246
Train Epoch: 1 [60000/60000 (100%)]	Losses 0: 0.211674 1: 0.159439
Test set:
0: Loss: 0.1569	Accuracy: 9541/10000 (95%)
1: Loss: 0.1565	Accuracy: 9555/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Losses 0: 0.094450 1: 0.141363
Train Epoch: 2 [10000/60000 (17%)]	Losses 0: 0.089772 1: 0.051544
Train Epoch: 2 [20000/60000 (33%)]	Losses 0: 0.219824 1: 0.220849
Train Epoch: 2 [30000/60000 (50%)]	Losses 0: 0.232826 1: 0.276150
Train Epoch: 2 [40000/60000 (67%)]	Losses 0: 0.176678 1: 0.159660
Train Epoch: 2 [50000/60000 (83%)]	Losses 0: 0.167689 1: 0.181849
Train Epoch: 2 [60000/60000 (100%)]	Losses 0: 0.042729